In [22]:
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Загрузка данных из JSON файла
with open('command_accuracy_report.json', 'r', encoding='utf-8') as f:
    report = json.load(f)

# Подготовка данных
summary = report['summary']
by_command = report['by_command']
details = report['details']

# Создаем DataFrame для визуализации
commands = list(by_command.keys())
accuracy_values = [float(by_command[cmd]['accuracy'].strip('%')) for cmd in commands]
correct_counts = [by_command[cmd]['correct'] for cmd in commands]
total_counts = [by_command[cmd]['total'] for cmd in commands]
error_counts = [total - correct for total, correct in zip(total_counts, correct_counts)]

# Создаем DataFrame для ошибок
errors = [item for item in details if not item['correct']]
error_commands = [item['command'] for item in errors]
error_types = [f"{item['expected']['команда']} vs {item['actual']['команда']}" for item in errors]
error_df = pd.DataFrame({
    'command': error_commands,
    'error_type': error_types,
    'expected': [str(item['expected']) for item in errors],
    'actual': [str(item['actual']) for item in errors]
})

# Создаем интерактивную визуализацию
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "indicator"}, {"type": "pie"}],
           [{"type": "bar"}, {"type": "bar"}]],
    subplot_titles=(
        "Общая точность распознавания", 
        "Распределение по командам",
        "Точность по типам команд",
        "Количество ошибок по типам команд"
    ),
    vertical_spacing=0.15,
    horizontal_spacing=0.1
)

# 1. Общая точность (индикатор)
fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=float(summary['accuracy'].strip('%')),
        gauge={
            'axis': {'range': [None, 100]},
            'bar': {'color': "darkblue"},
            'steps': [
                {'range': [0, 50], 'color': "red"},
                {'range': [50, 75], 'color': "orange"},
                {'range': [75, 100], 'color': "green"}
            ],
            'threshold': {
                'line': {'color': "black", 'width': 4},
                'thickness': 0.75,
                'value': float(summary['accuracy'].strip('%'))
            }
        }
    ),
    row=1, col=1
)

# 2. Распределение тестов по командам (pie chart)
fig.add_trace(
    go.Pie(
        labels=commands,
        values=total_counts,
        textinfo='percent+label',
        hole=0.3,
        hoverinfo='label+value+percent',
        showlegend=False,
        marker=dict(colors=px.colors.qualitative.Pastel)
    ),
    row=1, col=2
)

# 3. Точность по типам команд (bar chart)
fig.add_trace(
    go.Bar(
        x=commands,
        y=accuracy_values,
        text=[f"{acc:.1f}%" for acc in accuracy_values],
        textposition='auto',
        marker_color=px.colors.qualitative.Pastel,
        hovertext=[f"Правильно: {c}/{t}" for c, t in zip(correct_counts, total_counts)]
    ),
    row=2, col=1
)

# 4. Количество ошибок по командам (bar chart)
fig.add_trace(
    go.Bar(
        x=commands,
        y=error_counts,
        text=error_counts,
        textposition='auto',
        marker_color='#ef553b',
        hovertext=[f"Всего тестов: {t}" for t in total_counts]
    ),
    row=2, col=2
)

# Настройка макета
fig.update_layout(
    title_text=f'Анализ точности распознавания команд дрона<br><sup>Всего тестов: {summary["total_tests"]}, Время обработки: {summary["time_per_command"]} на команду</sup>',
    height=800,
    showlegend=False,
    template='plotly_white',
    margin=dict(t=120)
)

# Настройка осей
fig.update_yaxes(title_text="Точность (%)", range=[0, 100], row=2, col=1)
fig.update_yaxes(title_text="Количество ошибок", row=2, col=2)
fig.update_xaxes(title_text="Тип команды", row=2, col=1)
fig.update_xaxes(title_text="Тип команды", row=2, col=2)

# Дополнительные улучшения
fig.update_annotations(font_size=12)  # Размер шрифта подзаголовков

# Сохраняем в HTML файл
fig.write_html("command_accuracy_visualization1.html")

print("Визуализация сохранена в command_accuracy_visualization.html")

Визуализация сохранена в command_accuracy_visualization.html


In [31]:
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Загрузка данных из JSON файла
with open('command_accuracy_report.json', 'r', encoding='utf-8') as f:
    report = json.load(f)

# Подготовка данных
summary = report['summary']
by_command = report['by_command']
error_distribution = summary.get('error_distribution', {})
details = report.get('details', [])

# Создаем DataFrame для визуализации
commands = list(by_command.keys())
accuracy_values = [float(by_command[cmd]['accuracy'].strip('%')) for cmd in commands]
correct_counts = [by_command[cmd]['correct'] for cmd in commands]
total_counts = [by_command[cmd]['total'] for cmd in commands]
error_counts = [total - correct for total, correct in zip(total_counts, correct_counts)]

# Подготовка данных для ошибок
error_types = list(error_distribution.keys())
error_type_counts = list(error_distribution.values())

# Создаем DataFrame для деталей ошибок
if details:
    errors_df = pd.DataFrame(details)
    # Фильтруем только ошибки
    errors_df = errors_df[~errors_df['correct']].copy()
    
    # Извлекаем ожидаемые команды
    errors_df['expected_command'] = errors_df['expected'].apply(lambda x: x['команда'])
    errors_df['actual_command'] = errors_df['actual'].apply(lambda x: x.get('команда', 'unknown'))
    
    # Создаем категорию ошибки
    errors_df['error_category'] = errors_df.apply(
        lambda row: "Неправильная команда" 
        if row['expected_command'] != row['actual_command'] 
        else "Ошибка параметров", 
        axis=1
    )
    
    # Топ-10 проблемных команд
    top_errors = errors_df.groupby('expected_command').size().nlargest(10).reset_index(name='count')
    top_error_commands = top_errors['expected_command'].tolist()
    top_error_counts = top_errors['count'].tolist()
else:
    top_error_commands = []
    top_error_counts = []

# Создаем интерактивную визуализацию
fig = make_subplots(
    rows=3, cols=2,
    specs=[
        [{"type": "indicator", "colspan": 2}, None],
        [{"type": "bar"}, {"type": "pie"}],
        [{"type": "bar"}, {"type": "bar"}]
    ],
    subplot_titles=(
        "Общая точность распознавания", 
        "Точность по типам команд",
        "Распределение ошибок по типам",
        "Топ проблемных команд",
        "Распределение тестов по командам"
    ),
    vertical_spacing=0.1,
    horizontal_spacing=0.15,
    row_heights=[0.2, 0.4, 0.4]
)

# 1. Общая точность (индикатор)
fig.add_trace(
    go.Indicator(
        mode="gauge+number+delta",
        value=float(summary['accuracy'].strip('%')),
        delta={'reference': 90},
        title={'text': "Общая точность"},
        gauge={
            'axis': {'range': [0, 100], 'tickwidth': 1, 'tickcolor': "darkblue"},
            'bar': {'color': "#1f77b4"},
            'steps': [
                {'range': [0, 70], 'color': "#ff7f0e"},
                {'range': [70, 85], 'color': "#d8d86e"},
                {'range': [85, 100], 'color': "#2ca02c"}
            ],
            'threshold': {
                'line': {'color': "black", 'width': 4},
                'thickness': 0.75,
                'value': float(summary['accuracy'].strip('%'))
            }
        }
    ),
    row=1, col=1
)

# 2. Точность по типам команд (bar chart)
fig.add_trace(
    go.Bar(
        x=commands,
        y=accuracy_values,
        text=[f"{acc:.1f}%" for acc in accuracy_values],
        textposition='auto',
        marker_color=px.colors.qualitative.Pastel,
        hoverinfo='text',
        hovertext=[f"Точность: {acc:.1f}%<br>Правильно: {c}/{t}" 
                  for acc, c, t in zip(accuracy_values, correct_counts, total_counts)]
    ),
    row=2, col=1
)

# 3. Распределение ошибок по типам (pie chart)
if error_types:
    fig.add_trace(
        go.Pie(
            labels=error_types,
            values=error_type_counts,
            textinfo='percent+label',
            hole=0.4,
            hoverinfo='label+value+percent',
            marker=dict(colors=px.colors.qualitative.Pastel),
            showlegend=False
        ),
        row=2, col=2
    )
else:
    fig.add_trace(
        go.Scatter(x=[0.5], y=[0.5], text=["Нет данных об ошибках"], mode="text"),
        row=2, col=2
    )

# 4. Топ проблемных команд (bar chart)
if top_error_commands:
    fig.add_trace(
        go.Bar(
            x=top_error_commands,
            y=top_error_counts,
            text=top_error_counts,
            textposition='auto',
            marker_color='#ef553b',
            hoverinfo='y+text',
            hovertext=[f"Ошибок: {count}" for count in top_error_counts]
        ),
        row=3, col=1
    )
else:
    fig.add_trace(
        go.Scatter(x=[0.5], y=[0.5], text=["Нет данных об ошибках"], mode="text"),
        row=3, col=1
    )

# 5. Распределение тестов по командам (bar chart)
fig.add_trace(
    go.Bar(
        x=commands,
        y=total_counts,
        text=total_counts,
        textposition='auto',
        marker_color='#17becf',
        hoverinfo='text',
        hovertext=[f"Всего тестов: {t}<br>Ошибок: {e}" 
                  for t, e in zip(total_counts, error_counts)]
    ),
    row=3, col=2
)

# Настройка макета
title_text = (f'<b>Анализ точности распознавания команд дрона</b><br>'
             f'<sup>Всего тестов: {summary["total_tests"]} | '
             f'Точность: {summary["accuracy"]} | '
             f'Время: {summary["total_time"]} | '
             f'Среднее время: {summary["avg_time_per_command"]}/команда</sup>')

fig.update_layout(
    title_text=title_text,
    height=1000,
    showlegend=False,
    template='plotly_white',
    margin=dict(t=150, b=50),
    hoverlabel=dict(
        bgcolor="white",
        font_size=12,
        font_family="Rockwell"
    )
)

# Настройка осей
fig.update_yaxes(title_text="Точность (%)", range=[0, 100], row=2, col=1)
fig.update_yaxes(title_text="Количество ошибок", row=3, col=1)
fig.update_yaxes(title_text="Количество тестов", row=3, col=2)
fig.update_xaxes(title_text="Тип команды", row=2, col=1, tickangle=45)
fig.update_xaxes(title_text="Команда", row=3, col=1, tickangle=45)
fig.update_xaxes(title_text="Тип команды", row=3, col=2, tickangle=45)

# Дополнительные улучшения
fig.update_annotations(font_size=12)  # Размер шрифта подзаголовков

# Добавляем аннотации с метриками
fig.add_annotation(
    x=0.5, y=-0.15,
    xref="paper", yref="paper",
    text=f"Правильно распознано: {summary['passed_tests']} команд | "
         f"Ошибки: {summary['total_tests'] - summary['passed_tests']}",
    showarrow=False,
    font=dict(size=12)
)

# Сохраняем в HTML файл
fig.write_html("command_accuracy_visualization.html")

print("Визуализация сохранена в command_accuracy_visualization1.html")

Визуализация сохранена в command_accuracy_visualization1.html
